In [29]:
%reset-f
%load_ext autoreload
%autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy as sp
from scipy.optimize import curve_fit
from tools import *



In [37]:
country_name = "brazil"
outdir = "out_"+country_name+"/"
os.makedirs(outdir,exist_ok=True)
input_dir = "input/"+country_name+"/"

### Piecewise linear trajectory

In [38]:
def traj(x,s,t,a,m):
    #print("in",(x,s,t,u,a,m))
    u=x[-1]#forces u to be last value, this is equivalent siince we still chose a
    return extrap(x,[0,s,t,u],[0,0,a,max(m,a)]) #forces positive slope

### Data

In [39]:
#Reads abatement the data
df_in=pd.read_csv(input_dir+"abat.csv").set_index("year")

#Rescales the data
power = (np.floor(np.log10(df_in.max().max())/3))*3
print("scaling at x10^-"+str(power))
abat_xl=df_in * 10**(-power);

year=df_in.index.values
print(year)

#Cost data
macs = macs = pd.read_csv(input_dir+"mac_names.csv").set_index("measure")
    

scaling at x10^-0.0
[2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024
 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039
 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050]


In [40]:
#Output dataframe
df = pd.DataFrame(index=df_in.columns,columns=["speed","MAC","potential","t","growth"])
df["MAC"]=macs

### Loop on measures

In [41]:
for j in df.index:
#for j in ["Industrial wastewater"]:
    y=abat_xl.ix[:,j].interpolate() #fills missing values by interpolating

    #guess
    s_ = year[3];
    t_ = year[15];
    a_ = y[t_];
    m_ = y[year[-1]]
    
    #fit
    (s,t,a,m),foo =curve_fit(traj, year, y, maxfev=int(1e4), p0 = [s_, t_,a_,m_])
   
    #makes sure start is before t
    s,t = min(s,t),max(s,t)
    
    #plot
    plt.scatter(year,y,c="red")
    plt.plot(year,traj(year,s,t,a,m))
    
    u=year[-1]
    
    #speeds
    if abs(s-t)<2: #case with a jump
        va= a
    else:
        va = a/(t-s)
        
    vb = (m-a)/(u-t)
    
    if va>vb: #classic case
        df["t"]=t
        df["speed"]=va
        df["growth"]=vb
        df["potential"]=a
    else:
        v=m/(u-s)
        df["t"]=s
        df["speed"]=v
        df["growth"]=v
        df["potential"]=0
        
        plt.plot(year,traj(year,s,s,0,v*(u-s)),color="green")

    #saves figures and clears
    plt.savefig(outdir+j+".png")
    plt.close()
    